###  Note: this script is not working yet, because explainability is not directly supported for BlazingText. There is a workaround, but this will need to get the prioritized. See https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_processing/fairness_and_explainability/text_explainability_sagemaker_algorithm/byo_blazingtext_model_hosting.html

In [9]:
# Enable current type hints for older Python version (<3.10) 
from __future__ import annotations
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from pathlib import Path
import pickle
import logging
from dotenv import dotenv_values
import pdb
from pprint import pprint

import sagemaker


INPUT_MODE = 'File'  # File or Pipe
MAX_TRAIN_JOBS = 10
INSTANCE_TYPE_PREDICT = "ml.m5.xlarge"

config = dotenv_values(".env")
BUCKET = config['BUCKET']


logger = logging.getLogger(__name__)

sm_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')
role = sagemaker.get_execution_role()
session = boto3.Session()
region_name = session.region_name
image_uri = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


In [27]:
from sagemaker import clarify

INSTANCE_TYPE_CLARIFY = INSTANCE_TYPE_PREDICT
DATA_TYPE = "application/jsonlines"

# Configs
# =======
shap_config = clarify.SHAPConfig(
    save_local_shap_values=True, # Global aggregation is not supported yet
    text_config=clarify.TextConfig(
        language="english", 
        granularity="token", # tokens, sentences or paragraphs. 
    ),
    baseline=[["<UNK>"]],
)

data_config = clarify.DataConfig(
    s3_data_input_path=f's3://{BUCKET}/explainability-input',
    s3_output_path=f's3://{BUCKET}/explainability-output',
    dataset_type=DATA_TYPE,
    features='input_data',
    label='true_label',
    # predicted_label='p_reply',
)

# Load model name
with open('data/model_name.pickle', 'rb') as file:
    name_of_best_model = pickle.load(file=file)
model_config = clarify.ModelConfig(
    model_name=name_of_best_model,
    instance_type=INSTANCE_TYPE_CLARIFY,  
    instance_count=1
)

predictions_config = clarify.ModelPredictedLabelConfig(
    probability_threshold=0.5
)

# Run
# ====
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type=INSTANCE_TYPE_CLARIFY, 
    sagemaker_session=sagemaker.Session()
)
clarify_processor.run_explainability(
    data_config=data_config,
    model_config=model_config,
    explainability_config=shap_config,
    model_scores='p_reply',
)


Job Name:  Clarify-Explainability-2022-11-04-11-15-52-522
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://email-classification-sagemaker/explainability-input', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://email-classification-sagemaker/explainability-output/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://email-classification-sagemaker/explainability-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
...............................2022-11-04 11:20:55,227 logging.conf not found when

UnexpectedStatusException: Error for Processing job Clarify-Explainability-2022-11-04-11-15-52-522: Failed. Reason: ClientError: Expecting ',' delimiter: line 1 column 7112 (char 7111)